## COM361 &mdash; Introdução a Otimização &mdash; 2023, Prof. Amit ##

# Controle Ótimo de Epidemia #

#### Helena de Souza Douroucas (email), Pedro Nunes Pereira (pedronunes@poli.ufrj.br), Vinícius Quintanilha Porto Gomes (viniciusporto@poli.ufrj.br)

*****

### Índice

1. [Introdução](#1.-Introdução)
1. [Modelo Matemático](#2.-Modelo Mathemático)
1. [Solução](#3.-Solução)
1. [Resultados e Discussão](#4.-Resultados-e-discussão)
  1. [Subseção Opcional](#4.A.-Acrescente-subseções-se-necessário)
1. [Conclusão](#5.-Conclusão)
1. [Referências bibliográficas](#6.-Referências bibliográficas)

## 1. Introdução ##

Este trabalho busca realizar o controle ótimo de uma epidemia utilizando técnicas de otimização por meio do software Julia. Entre os anos de 2020 e 2022 a

As primeiras frases devem dar um panorama geral do projeto. Em seguida, uma descrição mais detalhada do problema que será resolvido, uma breve história do surgimento do problema (com [citations](https://en.wikipedia.org/wiki/Citation)) que será abordado, porque é importante e interessante e quaisquer outros fatos correlatos que queira expor. A fonte dos dados utilizados deve ser explicitada (pesquisa? internet? gerados sinteticamente?). Também deve ser feito um esboço do restante do relatório.


Esta seção deve ter entre 300 e 600 palavras, e **deve ser acessível a leigos** (não deve ser suposto que o leitor desta seção cursou esta disciplina). Podem incluir figuras se acharem convenientes:

![fixit flowchart][flow]

Para mais ajuda no uso de Markdown, veja [este link](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).

[flow]: https://s-media-cache-ak0.pinimg.com/736x/f5/75/c5/f575c53b93724808c6f0211890a54900.jpg

## 2. Modelo matemático ##

Esta seção deve conter uma discussão das hipóteses de modelagem feitas no problema (conforme a origem do problema: física? economia? redes sociais? ...). Explique a escolha das variáveis de decisão, as restrições e a função objetivo. Finalmente, mostre o problema de otimização escrite em forma padrão. Discute o tipo de modelo adotado (LP, QP, MIP, etc.). Equações devem ser formatadas em $\LaTeX$ dentro do notebook Julia. Nesta seção, pode supor que **o leitor está familiarizado com a matéria"**.

Eis um exemplo de uma equação:

$$
\begin{bmatrix}
  1 & 2 \\
  3 & 4
\end{bmatrix}
\begin{bmatrix} x \\ y \end{bmatrix} =
\begin{bmatrix} 5 \\ 6 \end{bmatrix}
$$

Aqui está um exemplo de um problema de otimização no formato padrão:

$$
\begin{aligned}
\underset{x \in \mathbb{R^n}}{\text{maximize}}\qquad& f_0(x) \\
\text{sujeito a:}\qquad& f_i(x) \le 0 && i=1,\dots,m\\
& h_j(x) = 0 && j=1,\dots,r
\end{aligned}
$$

Para algumas dicas rápidas sobre a utilizção de $\LaTeX$, veja [este cheat sheet](http://users.dickinson.edu/~richesod/latex/latexcheatsheet.pdf).

O modelo  divide a população em 3 grupos diferentes: sucetíveis (S), infectados (I) e recuperados (R).
O modelo usado por [GAFF] (trocar por uma referência) considera o caso contínuo e define as derivadas de S, I e R. Nesse trabalho, usaremos um modelo de tempo discreto, com unidade de tempo de um dia.

$$S[k+1] = S[k] + \mu N[k] - \beta \frac{S[k]I[k]}{N[k]} - u_1[k]S[k] - \mu \frac{N[k]S[k]}{K}$$

$$I[k+1] = I[k] + \beta \frac{S[k]I[k]}{N[k]} - (\delta + \gamma + u_2[k])I[k] - \mu \frac{N[k]I[k]}{K}$$

$$R[k+1] = R[k]- u_1[k]S[k] + (\gamma + u_2[k])I[k] - \mu \frac{N[k]R[k]}{K}$$

$%Inserir explicação para cada termo$

A função objetivo usada no caso contínuo é:

$$\int _0^{T_f} \left[B_1 I(t) + B_2 \left[\frac{R(t)}{K}\right]^m u_1^2(t) + B_3 u_2^2(t) \right] dt$$

Onde $B_1, B_2, B_3, m$ são parâmetros que vão alterar os pesos de cada termo.

Levando para o caso discreto temos:

$$\sum _{k=1}^{T_f} \left[B_1 I[k] + B_2 \left[\frac{R[k]}{K}\right]^m u_1^2[k] + B_3 u_2^2[k] \right]$$

O artigo adiciona mais duas restrições, $u_1(t) < 0.1$ e $u_2(t) < 0.6$, equivalente a $u_1[k] < 0.1$ e $u_2[k] < 0.6$. Essa escolha é equivalente a limitar a vacinação a no máximo 10% da população sucetível por dia e o tratamento a no máxima 60% por dia.

Dependendo da epidemia e da população que está sendo analisada (uma populosa capital de estado ou uma pequena cidade do interior), esses limites precisarão ser alterados. Também é presuposto que a vacinação está disponível desde o início do periodo analisado, o que pode não ser o caso. 

## 3. Solução ##

Nesta seção, coloque seu código em Julia + JuMP e resolva o problema proposto. Seu código deve ser limpo (não macarrônico!), de fácil leitura, bem comentado e anotado e deve compilar sem erros em Julia 1.x, x$\geq 1$! Não valem códigos em outras linguagens. **Vou rodar seu código para avaliar seu projeto**. Sugiro a utilização de múltiplos blocos de códigos separados por blocos de texto (células Markdown) explicando as várias partes da sua solução. Sugiro também a resolução de várias versões do seu problema, com modelos e hipóteses diferentes.

É permitido chamar pacotes externos, mas evite a utilização de bibliotecas exóticas (pois, em geral, não rodam em todas as versões de Julia, e terei que instalar a mesma versão que você usou, ou rodar na plataforma Google Colab, que gostaria de evitar).

In [3]:
# Este é um exemplo de um bloco de código
using JuMP, Clp
m = Model(with_optimizer(Clp.Optimizer,LogLevel=0) )
bichos = [:cavalos, :jegues, :cabras]  # estes são os bichos 
@variable(m, x[bichos] >= 0)          # as quantidades de cada um (não podem ser negativas)
@constraint(m, sum(x) <= 10)          # não podemos ter mais de 10 no total.
@objective(m, Max, x[:cavalos])        # queremos maximizar o número de cavalos
optimize!(m)

for i in bichos
    println("O número total de ", i, " é: ", JuMP.value.(x[i]))     # imprime o resultado na tela
end

O número total de cavalos é: 10.0
O número total de jegues é: 0.0
O número total de cabras é: 0.0


In [102]:
using JuMP, Ipopt, Plots


m = Model(Ipopt.Optimizer)
Tf = 50
mu, beta, gamma, delta, K = (0.00004 , 0.5, 0.1, 0.1, 5000)
B1, B2, B3 = (1, 1000, 1000)

@variable(m, S[1:Tf] >= 0)   #Número de Suscetiveis
@variable(m, I[1:Tf] >= 0)   #Número de Infectados
@variable(m, R[1:Tf] >= 0)   #Número de "Removidos"
@variable(m, N[1:Tf] >= 0)   #População total
@variable(m, u1[1:Tf] >= 0)
@variable(m, u2[1:Tf] >= 0)   #População total

#Condições iniciais
@constraint(m, S[1] == 4500) 
@constraint(m, I[1] == 499)
@constraint(m, R[1] == 1)


#Definindo Valores iniciais 
#set_start_value.(S[1], 4500)
#set_start_value.(I[1], 499)
#set_start_value.(R[1], 1)

#for i in 1:Tf
    #set_start_value.(x, 3)
    #@constraint(m, N[i] == S[i] + I[i] + R[i])
   # @constraint(m, u2[i] == 0.1)
#end

#for k in 1:Tf-1
   # @NLconstraint(m, S[k+1] == S[k] + mu*N[k] - beta*S[k]*I[k]/N[k] - mu*N[k]*S[k]/K)
   # @NLconstraint(m, I[k+1] == I[k] + beta*S[k]*I[k]/N[k] - (gamma + δ)*I[k] - mu*N[k]*I[k]/K)
  #  @constraint(m, R[k+1] == R[k] + (gamma)*I[k] - mu*N[k]*R[k]/K)
#end


for i in 1:Tf
    @constraint(m, N[i] == S[i] + I[i] + R[i])
    @constraint(m, u1[i] <= 0.1)
    @constraint(m, u2[i] <= 0.6)

    #@constraint(m, u1[i] == 0.1)
    #@constraint(m, u2[i] == 0.6)
end

for k in 1:Tf-1
    @NLconstraint(m, S[k+1]  >= S[k] - u1[k]*S[k] + mu*N[k] - beta*S[k]*I[k]/N[k] - mu*N[k]*S[k]/K)
    @NLconstraint(m, S[k+1]  <= S[k] - u1[k]*S[k] +  mu*N[k] - beta*S[k]*I[k]/N[k] - mu*N[k]*S[k]/K)
    #@NLconstraint(m, S[k+1] == S[k] + mu*N[k] - beta*S[k]*I[k]/N[k] - mu*N[k]*S[k]/K)

    #@NLconstraint(m, I[k+1] + 0.0 >= I[k] + beta*S[k]*I[k]/N[k] - (gamma + δ + u2[k])*I[k] - mu*N[k]*I[k]/K)
    #@NLconstraint(m, I[k+1] - 0.0 <= I[k] + beta*S[k]*I[k]/N[k] - (gamma + δ + u2[k])*I[k] - mu*N[k]*I[k]/K)
    @NLconstraint(m, I[k+1] == I[k] + beta*S[k]*I[k]/N[k] - (gamma + δ + u2[k])*I[k] - mu*N[k]*I[k]/K)

    @constraint(m, R[k+1] == R[k] + u1[k]*S[k] + (gamma + u2[k])*I[k] - mu*N[k]*R[k]/K)
end

#@objective(m, Min, 0)

@NLobjective(m, Min, sum(B1*I + (B2*(R/K).^2) .*u1.^2 + B3*u2.^2))
optimize!(m)

Suscetiveis = JuMP.value.(S)
Infectados = JuMP.value.(I)
Removidos = JuMP.value.(R)
Total = JuMP.value.(N)
println(JuMP.value.(u1))
println(JuMP.value.(u2))
println(JuMP.value.(I))
println("Soma dos infectados: ",sum(JuMP.value.(I)))
plot([Suscetiveis, Infectados, Removidos, Total])

LoadError: Unrecognized function ".^" used in nonlinear expression.

You must register it as a user-defined function before building
the model. For example, replacing `N` with the appropriate number
of arguments, do:
```julia
model = Model()
register(model, :.^, N, .^, autodiff=true)
# ... variables and constraints ...
```


**Tenha certeza de que seu código compila corretamente! Rodarei seu código!**

## 4. Resultados e discussão ##

Neste seção, os resultados obtidos serão exibidos e discutidos. Mostre figuras, gráficos, imagens, curvas de compromisso, e o que mais puder melhor ilustrar seus resultados. A discussão deverá explicar o que significam os resultados e como interpretá-los. As limitações da sua abordagem/modelo também devem ser colocadas, bem como uma análise da sensibilidade dos resultados em relação às hipóteses feitas.


Utilize plots (veja exemplos  `PyPlot` [aqui](https://gist.github.com/gizmaa/7214002))

Aqui está um exemplo de uma tabela (em Markdown):

| Tabelas        | São           | Boas  |
| ------------- |:-------------:| -----:|
| col 3 é      | alinhado à direita |\$1600 |
| col 2 é      | centrado      |  \$12 |
| texto | também serve      |   \$1 |

### 4.A. Subseções devem ser utilizadas para organizar seu texto.

#### 4.A.a. ou até subsubseções.

## 5. Conclusão ##

Faça um resumo do que encontrou e dos seus resultados, e fale de pelo menos uma direção na qual  seu trabalho pode ser desenvolvido no futuro, algo que poderia ser interessante em decorrência do seu projeto.


## 6. Referências bibliográficas ##

Nesta seção, cite _*todas*_ as referências utilizadas, na formulação matemática, no código ou para extrair dados ou figuras. Omissão de fontes é transgressão grave, denominada plágio.